In [1]:
# Imports
import torch
import torch.nn as nn
from torch.nn import functional as F
import os

In [2]:
# Set the device to 'mps' if CUDA is available, otherwise set it to 'cpu' for using apple silicon
device = "mps" if torch.backends.mps.is_available() else "cpu"
# Print the device that will be used for computations
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
# dropout = 0.2

mps


In [3]:
# Open the file 'wizard_of_oz.txt' in read mode with utf-8 encoding
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    # Read the entire content of the file into the variable 'text'
    text = f.read()

# Create a sorted set of unique characters present in the 'text'
chars = sorted(set(text))

# Calculate the size of the vocabulary, which is the number of unique characters in the 'text'
vocab_size = len(chars)

In [4]:
# Create a dictionary to map each character to a unique integer
string_to_int = { ch:i for i, ch in enumerate(chars) }
# Create a dictionary to map each integer back to its corresponding character
int_to_string = { i:ch for i, ch in enumerate(chars) }
# Define a function to encode a string into a list of integers
encode = lambda s: [string_to_int[c] for c in s]
# Define a function to decode a list of integers back into a string
decode = lambda l: ''.join([int_to_string[i] for i in l])
# Encode the entire text into a tensor of integers
data = torch.tensor(encode(text), dtype=torch.long)

In [5]:
# Calculate the number of elements that correspond to 80% of the total data length
n = int(0.8*len(data))

# Split the data into training and validation sets
train_data, val_data = data[:n], data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[58,  1, 73, 74, 60, 72,  9,  1],
        [72, 58, 57,  1, 68, 67, 58,  1],
        [56, 65, 62, 66, 55, 58, 57,  1],
        [54, 67, 54, 60, 58, 57,  9,  1]], device='mps:0')
targets:
tensor([[ 1, 73, 74, 60, 72,  9,  1, 50],
        [58, 57,  1, 68, 67, 58,  1, 68],
        [65, 62, 66, 55, 58, 57,  1, 62],
        [67, 54, 60, 58, 57,  9,  1, 55]], device='mps:0')


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
# # Set the block size to 8. This is the number of elements to consider at a time from the training data.
# block_size = 8

# # Split the training data into two parts: 'x' and 'y'. 'x' is the first 'block_size' elements and 'y' is the next 'block_size' elements.
# x, y = train_data[:block_size],  train_data[1:block_size+1]

# # Loop over the range of block_size
# for t in range(block_size):
#     # The context is the first 't+1' elements of 'x'
#     context = x[:t+1]
#     # The target is the 't'-th element of 'y'
#     target = y[t]
#     # Print the context and target
#     print('when input is', context, 'target is', target)

In [8]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)

        return index

In [9]:
model = torch.load("bigram.pth") if os.path.exists("bigram.pth") else BigramLanguageModel(vocab_size) 
m = model.to(device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


 m sharcofthed.

ant ve afoucamo ted. s ceroo the?"N whel med

wom. tyor Ply lfid h
Thond uemilarasth ted we.]Fje y ta
wiviggng, s thttadise anghe ully and Thony. asheo a s."I's Ze as thelagasthenn," lemaroyo ladis he he y, mer trstorat an iland, forrd  tomersiore gicuthe tthethou " adonowalenotthe tse thevaio lurne o. Theringlly they tore ache arelebucuche theryr tesireshorouf u pr tathyere, bed hearilly siedioweyouro mougimy ind thede g tide towe  lilowathettrure a them toniny. are


yopre wng


In [10]:
%%time
# Create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.434, val loss: 2.519
step: 250, train loss: 2.440, val loss: 2.508
step: 500, train loss: 2.443, val loss: 2.462
step: 750, train loss: 2.441, val loss: 2.475
step: 1000, train loss: 2.462, val loss: 2.467
step: 1250, train loss: 2.449, val loss: 2.463
step: 1500, train loss: 2.417, val loss: 2.455
step: 1750, train loss: 2.435, val loss: 2.489
step: 2000, train loss: 2.441, val loss: 2.463
step: 2250, train loss: 2.426, val loss: 2.473
step: 2500, train loss: 2.416, val loss: 2.480
step: 2750, train loss: 2.415, val loss: 2.446
step: 3000, train loss: 2.438, val loss: 2.478
step: 3250, train loss: 2.449, val loss: 2.481
step: 3500, train loss: 2.419, val loss: 2.450
step: 3750, train loss: 2.433, val loss: 2.461
step: 4000, train loss: 2.427, val loss: 2.512
step: 4250, train loss: 2.420, val loss: 2.479
step: 4500, train loss: 2.414, val loss: 2.497
step: 4750, train loss: 2.456, val loss: 2.488
step: 5000, train loss: 2.412, val loss: 2.480
step: 5250, train l

In [11]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


n med oncle
m thending, len at
Thy, be sursthet?" tote p vetoreeis
" s anctlid wioy the nd tovinkeraco bls mantig, hathe ucouithay auithed "Whtheng wain wizaniothe erind orally inofo b s pos. wndeerearrerd w am,"
pthiglooksthad thinddnens, nd h E
atinond th ikilld sckiopemerdopoo
"Burrse thed gremo ly an s Th, usealkne tougrwan ancen ftub-sooat tidinenong lar igaige thesthte lsunithache the tand t w Do GE thecace 5A th cea n edss lt we ima   whero Mangedise."
"NKS4f umarsa anoure coofroucuppovev


In [12]:
torch.save(m, "bigram.pth")